In [ ]:
import numpy as np
from sympy import MatrixSymbol, Matrix, poly
# x^T Q x +

In [ ]:
# Example Formulation of TSP tour length and setting up TSP function by testing random permutations as solutions. Extremely inefficient.
n = 4
distanceArr = [[0, 10, 50, 45], [10, 0, 25, 25], [50, 25, 0, 40], [45, 25, 40, 0]]

def traveling_salesmen(x, dist):
    return sum(dist[x[i]][x[(i+1) % n]] for i in range(n))

x = [0, 2, 3, 1]

lengthOfRoute = traveling_salesmen(x, distanceArr)
print("Given the arrangement of cities, return the route distance of all of those cities.:", lengthOfRoute)

Given the arrangement of cities, return the route distance of all of those cities.: 125


In [ ]:
# Adjacency List storage between Nodes.
adjList = np.array([
    [0, 10, 50, 45],
    [10, 0, 25, 25],
    [50, 25, 0, 40],
    [45, 25, 40, 0]
])

In [ ]:
# Edge Collector. Returns aall of the edges from the Adjacency List and their weights
edges = []
n = adjList.shape[0]
for i in range(n):
    for j in range(i + 1, n):
        edges.append((i, j, adjList[i, j]))

# Sort edges by Weights.
sorted_edges = sorted(edges, key=lambda x: x[2])

# Sort edges by Weight in decreasing order.
sorted_edges_desc = sorted(edges, key=lambda x: x[2], reverse=True)


# In order to figure out constraints A, B, C we must calculate the n largest edges minus the n smallest edges + 1 to return those constraints values.
# This constraints values guarantee that any working solution will have an lower objective function than any faulty solutions.

# Selecting the smallest edges for the LB (lower bound). Four edges.
LB_edges = sorted_edges[:4]

# The Lower Bound is equal to all of these edges added up.
LB = sum(edge[2] for edge in LB_edges)

# Selecting the smallest edges for the UB (Upper bound). Four edges.
UB_edges = sorted_edges_desc[:4]

# The Upper Bound is equal to all of these edges added up.
UB = sum(edge[2] for edge in UB_edges)

# Display the results
print("Lower Bound Edges (LB):", LB_edges)
print("Lower Bound (LB):", LB)

# Display the results
print("Upper Bound Edges (UB):", UB_edges)
print("Upper Bound (UB):", UB)
print(UB - LB + 1)

# A, B, C Solution
A = B = C = UB - LB + 1

Lower Bound Edges (LB): [(0, 1, np.int64(10)), (1, 2, np.int64(25)), (1, 3, np.int64(25)), (2, 3, np.int64(40))]
Lower Bound (LB): 100
Upper Bound Edges (UB): [(0, 2, np.int64(50)), (0, 3, np.int64(45)), (2, 3, np.int64(40)), (1, 2, np.int64(25))]
Upper Bound (UB): 160
61


In [ ]:
# All of these constraints operations must be set equal to 0 therefore the minus 1.
#https://github.com/amoraglio/quantum-optimization-tutorial/blob/main/QUBO%20Formulation%20of%20Travelling%20Salesman%20Problem.ipynb
# Created custom sum of edges function for any adjList array.

def objective_function(x, y, dist):
    return \
    \
    sum(dist[i][j] * x[i, j] for i in range(len(dist)) for j in range(len(dist)) if i != j) + \
    \
    A * (x[1,0] + x[2,0] + x[3,0] - 1)**2 + \
    A * (x[0,1] + x[2,1] + x[3,1] - 1)**2 + \
    A * (x[0,2] + x[1,2] + x[3,2] - 1)**2 + \
    A * (x[0,3] + x[1,3] + x[2,3] - 1)**2 + \
    \
    B * (x[0,1] + x[0,2] + x[0,3] - 1)**2 + \
    B * (x[1,0] + x[1,2] + x[1,3] - 1)**2 + \
    B * (x[2,0] + x[2,1] + x[2,3] - 1)**2 + \
    B * (x[3,0] + x[3,1] + x[3,2] - 1)**2 + \
    \
    C * x[1,1]**2 + \
    C * x[2,2]**2 + \
    C * x[3,3]**2 + \
    C * (x[1,1] + x[1,2] + x[2,1] + x[2,2] + y[0] - 1)**2 + \
    C * (x[1,1] + x[1,3] + x[3,1] + x[3,3] + y[1] - 1)**2 + \
    C * (x[2,2] + x[2,3] + x[3,2] + x[3,3] + y[2] - 1)**2 + \
    C * (x[1,1] + x[1,2] + x[1,3] + x[2,1] + x[2,2] + x[2,3] + x[3,1] + x[3,2] + x[3,3] + y[3] + y[4] - 2)**2


# Create a random Binary Array with row length n and column length n.
x = np.random.choice([0, 1], size=(n, n))

# Create a random Binary Array of possible choices to choose.
y = np.random.choice([0, 1], size=(5))

# Call the Objective function.
x, y, objective_function(x, y, adjList)

(array([[0, 0, 0, 1],
        [0, 0, 0, 0],
        [0, 0, 1, 1],
        [1, 1, 1, 1]]),
 array([1, 1, 1, 1, 0]),
 np.int64(2940))

In [ ]:
X = MatrixSymbol('X', n, n)
Y = MatrixSymbol('Y', 1, 5)
sym_objective_function = poly(objective_function(X, Y, adjList))
sym_objective_function

Poly(122*X[0, 1]**2 + 122*X[0, 1]*X[0, 2] + 122*X[0, 1]*X[0, 3] + 122*X[0, 1]*X[2, 1] + 122*X[0, 1]*X[3, 1] - 234*X[0, 1] + 122*X[0, 2]**2 + 122*X[0, 2]*X[0, 3] + 122*X[0, 2]*X[1, 2] + 122*X[0, 2]*X[3, 2] - 194*X[0, 2] + 122*X[0, 3]**2 + 122*X[0, 3]*X[1, 3] + 122*X[0, 3]*X[2, 3] - 199*X[0, 3] + 122*X[1, 0]**2 + 122*X[1, 0]*X[1, 2] + 122*X[1, 0]*X[1, 3] + 122*X[1, 0]*X[2, 0] + 122*X[1, 0]*X[3, 0] - 234*X[1, 0] + 244*X[1, 1]**2 + 244*X[1, 1]*X[1, 2] + 244*X[1, 1]*X[1, 3] + 244*X[1, 1]*X[2, 1] + 244*X[1, 1]*X[2, 2] + 122*X[1, 1]*X[2, 3] + 244*X[1, 1]*X[3, 1] + 122*X[1, 1]*X[3, 2] + 244*X[1, 1]*X[3, 3] + 122*X[1, 1]*Y[0, 0] + 122*X[1, 1]*Y[0, 1] + 122*X[1, 1]*Y[0, 3] + 122*X[1, 1]*Y[0, 4] - 488*X[1, 1] + 244*X[1, 2]**2 + 244*X[1, 2]*X[1, 3] + 244*X[1, 2]*X[2, 1] + 244*X[1, 2]*X[2, 2] + 122*X[1, 2]*X[2, 3] + 122*X[1, 2]*X[3, 1] + 244*X[1, 2]*X[3, 2] + 122*X[1, 2]*X[3, 3] + 122*X[1, 2]*Y[0, 0] + 122*X[1, 2]*Y[0, 3] + 122*X[1, 2]*Y[0, 4] - 585*X[1, 2] + 244*X[1, 3]**2 + 122*X[1, 3]*X[2, 1] + 